In [3]:
import os
import sqlite3
import requests
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Configuration
BASE_DIRECTORY = r"C:\Users\howar\football_research_advanced\2.0\England\Football_Predictions\data"
DATABASE_PATH = os.path.join(BASE_DIRECTORY, "football_data.db")
BASE_URL = "https://v3.football.api-sports.io"
API_KEY = os.getenv("x-rapidapi-key")

class TeamVisualsLoader:
    def __init__(self):
        self.conn = sqlite3.connect(DATABASE_PATH)
        self.cursor = self.conn.cursor()
        self.headers = {
            "x-apisports-key": API_KEY
        }

    def initialize_table(self):
        """Create the team_visuals table if it doesn't exist."""
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS team_visuals (
                team_id INTEGER PRIMARY KEY,
                team_name TEXT,
                team_logo TEXT,
                league_id INTEGER,
                season INTEGER,
                last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        self.conn.commit()

    def fetch_team_data(self, league_id, season):
        """Fetch team data from the API."""
        try:
            print(f"Fetching team data for league {league_id}, season {season}...")
            response = requests.get(
                f"{BASE_URL}/teams",
                headers=self.headers,
                params={"league": league_id, "season": season}
            )
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching team data: {e}")
            return None

    def process_team_data(self, league_id, season):
        """Process and store team data in the database."""
        team_data = self.fetch_team_data(league_id, season)
        if not team_data or "response" not in team_data:
            print(f"No team data available for league {league_id}, season {season}.")
            return

        for team in team_data["response"]:
            team_id = team["team"]["id"]
            team_name = team["team"]["name"]
            team_logo = team["team"]["logo"]

            # Insert or update the team data
            self.cursor.execute('''
                INSERT OR REPLACE INTO team_visuals (team_id, team_name, team_logo, league_id, season, last_updated)
                VALUES (?, ?, ?, ?, ?, CURRENT_TIMESTAMP)
            ''', (team_id, team_name, team_logo, league_id, season))
            print(f"Stored visual data for team: {team_name} (ID: {team_id})")
        
        self.conn.commit()

    def close_connection(self):
        """Close the database connection."""
        self.conn.close()
        print("Database connection closed.")

if __name__ == "__main__":
    # Initialize the loader
    loader = TeamVisualsLoader()
    loader.initialize_table()

    # English leagues and season
    leagues = {
        "Premier League": 39,
        "Championship": 40,
        "League One": 41,
        "League Two": 42
    }
    season = 2024

    # Process team visuals for each league
    for league_name, league_id in leagues.items():
        print(f"Processing {league_name} (League ID: {league_id})...")
        loader.process_team_data(league_id, season)

    loader.close_connection()


Processing Premier League (League ID: 39)...
Fetching team data for league 39, season 2024...
Stored visual data for team: Manchester United (ID: 33)
Stored visual data for team: Newcastle (ID: 34)
Stored visual data for team: Bournemouth (ID: 35)
Stored visual data for team: Fulham (ID: 36)
Stored visual data for team: Wolves (ID: 39)
Stored visual data for team: Liverpool (ID: 40)
Stored visual data for team: Southampton (ID: 41)
Stored visual data for team: Arsenal (ID: 42)
Stored visual data for team: Everton (ID: 45)
Stored visual data for team: Leicester (ID: 46)
Stored visual data for team: Tottenham (ID: 47)
Stored visual data for team: West Ham (ID: 48)
Stored visual data for team: Chelsea (ID: 49)
Stored visual data for team: Manchester City (ID: 50)
Stored visual data for team: Brighton (ID: 51)
Stored visual data for team: Crystal Palace (ID: 52)
Stored visual data for team: Brentford (ID: 55)
Stored visual data for team: Ipswich (ID: 57)
Stored visual data for team: Nottin